# Account Management

## Install

In [1]:
SYFT_VERSION = ">=0.8.2.b0,<0.9"
package_string = f'"syft{SYFT_VERSION}"'
# %pip install {package_string} -f https://whls.blob.core.windows.net/unstable/index.html

In [2]:
import syft as sy
sy.requires(SYFT_VERSION)

✅ The installed version of syft==0.8.2b2 matches the requirement >=0.8.2b0 and the requirement <0.9


In [3]:
node = sy.orchestra.launch(name="account-management-example-domain-1", port=8041, reset=True)

Starting account-management-example-domain-1 server on 0.0.0.0:8041


Waiting for server to start Done.


## Setup

Lets login with our root user

In [4]:
from syft.service.user.user import UserUpdate, UserCreate, ServiceRole
client = node.login(email="info@openmined.org", password="changethis")

Logged into account-management-example-domain-1 as <info@openmined.org>


## Creating a User

We can create/get/update/delete users using the `user service`, which we can access via `client.api.services.user`. Lets create a new `User`

In [5]:
client.users.create(UserCreate(email="newuser@openmined.org", name="John Doe", password="pw"))

```python
class UserView:
  id: str = 5559e26947464008ae242b9136634b6a
  name: str = "John Doe"
  email: str = "newuser@openmined.org"
  institute: str = "<Missing>"
  website: str = None
  role: str = ServiceRole.DATA_SCIENTIST

```

## Getting users & inspecting roles

Lets query all our users, we can use `client.api.services.user.get_all` or simply `client.api.services.user`

In [6]:
users = client.users
users

We see 2 users, the root user which exists by default, and the user we just created.

In [7]:
new_user = [u for u in users if u.email == "newuser@openmined.org"][0]

We can view the new user, and see its permissions

In [8]:
new_user

```python
class UserView:
  id: str = 5559e26947464008ae242b9136634b6a
  name: str = "John Doe"
  email: str = "newuser@openmined.org"
  institute: str = "<Missing>"
  website: str = None
  role: str = ServiceRole.DATA_SCIENTIST

```

## Updating a User

Lets update the user we just created, and change the role using the `user.update` service method

In [9]:
# TODO explain how the Update objects work regarding missing fields etc

In [10]:
updated_user = client.users.update(new_user.id, 
    UserUpdate(role=ServiceRole.DATA_SCIENTIST, password="123")
)

In [11]:
updated_user

```python
class UserView:
  id: str = 5559e26947464008ae242b9136634b6a
  name: str = "John Doe"
  email: str = "newuser@openmined.org"
  institute: str = "<Missing>"
  website: str = None
  role: str = ServiceRole.DATA_SCIENTIST

```

We can now log in with our new user and run some query, which in this case returns an empty result

In [12]:
ds_client = node.login(email="newuser@openmined.org", password="123")

Logged into account-management-example-domain-1 as <newuser@openmined.org>


In [13]:
ds_client.datasets.get_all()

[]

## Deleting a User

Lastly, we can delete users using the `user.delete` service method

In [14]:
client.users.delete(new_user.id)

<class 'syft.service.response.SyftSuccess'>: ID: 5559e26947464008ae242b9136634b6a deleted

In [15]:
users = client.users
users

## Register Control

In [16]:
client.register(email="joker@test.com", password="joker123", name="Joker")

<class 'syft.service.response.SyftSuccess'>: User 'Joker' successfully registered! To see users, run `[your_client].users`

In [17]:
new_user = node.login(email="joker@test.com", password="joker123")

Logged into account-management-example-domain-1 as <joker@test.com>


In [18]:
client.users

By default, only root user can register new users

In [19]:
new_user.register(email="batman@test.com", password="batman123", name="Batman")

<class 'syft.service.response.SyftSuccess'>: User 'Batman' successfully registered!

If the root user enables guest users to register new users, then they can

In [20]:
client.settings.allow_guest_signup(enable=True)

<class 'syft.service.response.SyftSuccess'>: Registration feature successfully enabled

In [21]:
new_user.register(email="batman@test.com", password="batman123", name="Batman")

<class 'syft.service.response.SyftError'>: User already exists with email: batman@test.com

In [22]:
client.users

Now if the root user disables the register function, then only the root user can register new user

In [23]:
client.settings.allow_guest_signup(enable=False)

<class 'syft.service.response.SyftSuccess'>: Registration feature successfully disabled

In [24]:
new_user.register(email="harley@test.com", password="harley123", name="Harley")

<class 'syft.service.response.SyftError'>: You don't have permission to create an account on the domain: account-management-example-domain-1. Please contact the Domain Owner.

In [25]:
client.register(email="robin@test.com", password="robin123", name="Robin")

<class 'syft.service.response.SyftSuccess'>: User 'Robin' successfully registered! To see users, run `[your_client].users`

In [26]:
client.users